# Python 操作 MongoDB

In [19]:
import pymongo

## 连接数据库 Making a Connection with MongoClient

几种连接方法

- 简写 client = MongoClient()
- client = MongoClient('localhost',27017)
- 使用URI client = MongoClient('mongodb://localhost:27017/')

The first step when working with PyMongo is to create a :class:`~pymongo.mongo_client.MongoClient` to the running mongod instance. Doing so is easy:

In [20]:
from pymongo import MongoClient
client = MongoClient()

In [21]:
client.PORT

27017

In [22]:
client.HOST

'localhost'

The above code will connect on the default host and port. We can also specify the host and port explicitly, as follows:

In [23]:
client = MongoClient('localhost', 27017)

Or use the MongoDB URI format:

In [24]:
client = MongoClient('mongodb://localhost:27017/')

## Getting a Database

In [25]:
client.list_database_names()

['admin', 'config', 'local', 'student', 'test-database']

In [26]:
db = client.student

If your database name is such that using attribute style access won't work (like test-database), you can use dictionary style access instead:


In [27]:
db= client['student']

## Getting a Collection

A collection is a group of documents stored in MongoDB, and can be thought of as roughly the equivalent of a table in a relational database. Getting a collection in PyMongo works the same as getting a database:

In [28]:
db.list_collection_names()

['info_student']

In [29]:
collection = db.info_student

An important note about collections (and databases) in MongoDB is that they are created lazily - none of the above commands have actually performed any operations on the MongoDB server. Collections and databases are created when the first document is inserted into them.

## Documents

## 新增数据

In [30]:
from datetime import datetime

In [31]:
db = client['test-database']

In [32]:
import datetime
post = {"author": "Mike",
        "text": "My first blog post!",
        "tags": ["mongodb", "python", "pymongo"],
        "date": datetime.datetime.utcnow()}

In [33]:
posts = db.posts

In [34]:
post_id = posts.insert_one(post).inserted_id

In [35]:
post_id

ObjectId('5cf2339dd422c5028e526b1e')

In [36]:
db.list_collection_names()

['posts', 'info_student']

In [37]:
client.list_database_names()

['admin', 'config', 'local', 'student', 'test-database']

## 查询数据 Getting a Single Document With find_one()

In [38]:
client.list_database_names()

['admin', 'config', 'local', 'student', 'test-database']

In [39]:
db = client.student

In [40]:
db.list_collection_names()

['info_student']

In [41]:
# 查询一条数据

db.info_student.find_one()

{'_id': ObjectId('5cd599af54507e8759dba40a'),
 'name': 'bob',
 'age': 16.0,
 'sex': 'male',
 'grade': 95.0}

In [42]:
type(db.info_student.find_one())

dict

In [43]:
oid = db.info_student.find_one()['_id']

In [44]:
type(db.info_student.find({'sex':'female'},{'_id':0}))

pymongo.cursor.Cursor

In [45]:
for i in db.info_student.find({'sex':'female'},{'_id':0}):
    print(i)

{'name': 'ahb', 'age': 20.0, 'sex': 'female', 'grade': 45.0}
{'name': 'bob4', 'age': 18.0, 'sex': 'female', 'grade': 95.0}
{'name': '_新的标题_', 'age': 99, 'sex': 'female', 'created_at': datetime.datetime(2019, 5, 11, 23, 6, 58, 363000)}
{'name': '_新的标题_', 'age': 121, 'sex': 'female', 'created_at': datetime.datetime(2019, 5, 11, 23, 19, 51, 529000)}
{'name': '_新的标题_', 'age': 22, 'sex': 'female', 'created_at': datetime.datetime(2019, 5, 11, 23, 20, 43, 987000)}
{'name': '_新的标题_', 'age': 22, 'sex': 'female', 'created_at': datetime.datetime(2019, 5, 15, 23, 28, 49, 980000), 'tages': ['baseball', 'basketball']}
{'name': '_新的标题_', 'age': 22, 'sex': 'female', 'created_at': datetime.datetime(2019, 5, 15, 23, 36, 12, 274000), 'tages': ['swimming', 'basketball']}


In [46]:
db.info_student.find_one({"_id":oid})

{'_id': ObjectId('5cd599af54507e8759dba40a'),
 'name': 'bob',
 'age': 16.0,
 'sex': 'male',
 'grade': 95.0}

In [47]:
from bson.objectid import ObjectId

# The web framework gets post_id from the URL and passes it as a string
def get(post_id):
    # Convert from string to ObjectId:
    document = client.db.collection.find_one({'_id': ObjectId(post_id)})

In [48]:
db.info_student.find_one({"_id":ObjectId('5cd599af54507e8759dba40a')})

{'_id': ObjectId('5cd599af54507e8759dba40a'),
 'name': 'bob',
 'age': 16.0,
 'sex': 'male',
 'grade': 95.0}

In [49]:
db.info_student.find_one({'sex':'male'})

{'_id': ObjectId('5cd599af54507e8759dba40a'),
 'name': 'bob',
 'age': 16.0,
 'sex': 'male',
 'grade': 95.0}

In [50]:
import pprint

In [51]:
pprint.pprint(db.info_student.find_one({'sex':'female'}))

{'_id': ObjectId('5cd599c154507e8759dba40b'),
 'age': 20.0,
 'grade': 45.0,
 'name': 'ahb',
 'sex': 'female'}


## Bulk Inserts

In [57]:
client.list_database_names()

['admin', 'config', 'local', 'student', 'test-database']

In [60]:
db = client['test-database']

In [61]:
db.list_collection_names()

['posts', 'info_student']

In [63]:
posts = db.posts

In [64]:
new_posts = [{"author": "Mike",
...               "text": "Another post!",
...               "tags": ["bulk", "insert"],
...               "date": datetime.datetime(2009, 11, 12, 11, 14)},
...              {"author": "Eliot",
...               "title": "MongoDB is fun",
...               "text": "and pretty easy too!",
...               "date": datetime.datetime(2009, 11, 10, 10, 45)}]

In [65]:
result = posts.insert_many(new_posts)

In [66]:
result.inserted_ids

[ObjectId('5cf23447d422c5028e526b1f'), ObjectId('5cf23447d422c5028e526b20')]

In [74]:
for post in posts.find({'author':'Mike'}):
    print(post)

{'_id': ObjectId('5cdc32cad422c513f1369781'), 'author': 'Mike', 'text': 'My first blog post!', 'tags': ['mongodb', 'python', 'pymongo'], 'date': datetime.datetime(2019, 5, 15, 15, 39, 5, 162000)}
{'_id': ObjectId('5cdc332dd422c513f1369782'), 'author': 'Mike', 'text': 'My first blog post!', 'tags': ['mongodb', 'python', 'pymongo'], 'date': datetime.datetime(2019, 5, 15, 15, 41, 32, 257000)}
{'_id': ObjectId('5cdc3445d422c513f1369787'), 'author': 'Mike', 'text': 'My first blog post!', 'tags': ['mongodb', 'python', 'pymongo'], 'date': datetime.datetime(2019, 5, 15, 15, 45, 51, 478000)}
{'_id': ObjectId('5cf2339dd422c5028e526b1e'), 'author': 'Mike', 'text': 'My first blog post!', 'tags': ['mongodb', 'python', 'pymongo'], 'date': datetime.datetime(2019, 6, 1, 8, 13, 17, 42000)}
{'_id': ObjectId('5cf23447d422c5028e526b1f'), 'author': 'Mike', 'text': 'Another post!', 'tags': ['bulk', 'insert'], 'date': datetime.datetime(2009, 11, 12, 11, 14)}


## Count

In [71]:
posts.count_documents({})

6

In [72]:
posts.count_documents({'author':'Mike'})

5

## update

In [4]:
from pymongo import MongoClient

In [5]:
client = MongoClient()

In [7]:
client.list_database_names()

['Book_DouBan', 'admin', 'config', 'local', 'student', 'test-database']

In [24]:
db = client['test-database']

In [25]:
db.list_collection_names()

['posts', 'info_student']

In [76]:
rest = db.posts.update_one({'author':'Mike'},{"$set":{'author':"Joey"}})

In [77]:
rest.matched_count

1

In [78]:
rest.modified_count

1

In [90]:
rest = db.posts.update_many({'author':'Mike'},
                            {"$set":{'text':"it is last update"}})

In [91]:
rest.matched_count

2

In [92]:
rest.modified_count

2

In [94]:
rest = db.posts.update_many({},
                            {"$set":{'text':"it is last update"}})

In [95]:
rest.matched_count

6

In [96]:
rest.modified_count

4

In [97]:
for i in db.posts.find({}):
    print (i)

{'_id': ObjectId('5cdc32cad422c513f1369781'), 'text': 'it is last update'}
{'_id': ObjectId('5cdc332dd422c513f1369782'), 'author': 'Joey', 'text': 'it is last update', 'tags': ['mongodb', 'python', 'pymongo'], 'date': datetime.datetime(2019, 5, 15, 15, 41, 32, 257000)}
{'_id': ObjectId('5cdc3445d422c513f1369787'), 'author': 'Joey', 'text': 'it is last update', 'tags': ['mongodb', 'python', 'pymongo'], 'date': datetime.datetime(2019, 5, 15, 15, 45, 51, 478000)}
{'_id': ObjectId('5cf2339dd422c5028e526b1e'), 'author': 'Mike', 'text': 'it is last update', 'tags': ['mongodb', 'python', 'pymongo'], 'date': datetime.datetime(2019, 6, 1, 8, 13, 17, 42000)}
{'_id': ObjectId('5cf23447d422c5028e526b1f'), 'author': 'Mike', 'text': 'it is last update', 'tags': ['bulk', 'insert'], 'date': datetime.datetime(2009, 11, 12, 11, 14)}
{'_id': ObjectId('5cf23447d422c5028e526b20'), 'author': 'Eliot', 'title': 'MongoDB is fun', 'text': 'it is last update', 'date': datetime.datetime(2009, 11, 10, 10, 45)}


In [101]:
db = client.student

In [108]:
db.list_collection_names()

['info_student']

In [121]:
for i in db['info_student'].find({'name':'xi'}):
    print (i)

{'_id': ObjectId('5cd59a4954507e8759dba40c'), 'name': 'xi', 'age': 15.0, 'sex': 'male', 'grade': 75.0}
{'_id': ObjectId('5cd59a4954507e8759dba40d'), 'name': 'xi', 'age': 15.0, 'sex': 'male', 'grade': 75.0}
{'_id': ObjectId('5cd59a8954507e8759dba40e'), 'name': 'xi', 'age': 15.0, 'sex': 'male', 'grade': 75.0}
{'_id': ObjectId('5cd59a8954507e8759dba410'), 'name': 'xi', 'age': 15.0, 'sex': 'male', 'grade': 35.0}
{'_id': ObjectId('5cd59a8954507e8759dba411'), 'name': 'xi', 'age': 15.0, 'sex': 'male', 'grade': 25.0, 'school': 'soft'}
{'_id': ObjectId('5cd59aa054507e8759dba412'), 'name': 'xi', 'age': 15.0, 'sex': 'male', 'grade': 75.0}
{'_id': ObjectId('5cd59aa054507e8759dba413'), 'name': 'xi', 'age': 15.0, 'sex': 'male', 'grade': 75.0}


In [131]:
result = db['info_student'].update_many({'name':'xi'},{"$inc":{"age":10}})

In [132]:
result.matched_count

7

In [133]:
result.modified_count

7

## Delete

In [136]:
rest = student.delete_one({'name':'xi'})

In [137]:
rest.deleted_count

1

In [138]:
rest.raw_result

{'n': 1, 'ok': 1.0}

In [145]:
rest = student.delete_many({'age':{'$lt':20}})

In [146]:
rest.deleted_count

4

In [149]:
rest = student.delete_many({'age':{'$gt':90}})
rest.deleted_count

2

In [150]:
for i in student.find({}):
    print(i)

{'_id': ObjectId('5cd599c154507e8759dba40b'), 'name': 'ahb', 'age': 20.0, 'sex': 'female', 'grade': 45.0}
{'_id': ObjectId('5cd59a4954507e8759dba40d'), 'name': 'xi', 'age': 35.0, 'sex': 'male', 'grade': 75.0}
{'_id': ObjectId('5cd59a8954507e8759dba40e'), 'name': 'xi', 'age': 35.0, 'sex': 'male', 'grade': 75.0}
{'_id': ObjectId('5cd59a8954507e8759dba410'), 'name': 'xi', 'age': 35.0, 'sex': 'male', 'grade': 35.0}
{'_id': ObjectId('5cd59a8954507e8759dba411'), 'name': 'xi', 'age': 35.0, 'sex': 'male', 'grade': 25.0, 'school': 'soft'}
{'_id': ObjectId('5cd59aa054507e8759dba412'), 'name': 'xi', 'age': 35.0, 'sex': 'male', 'grade': 75.0}
{'_id': ObjectId('5cd59aa054507e8759dba413'), 'name': 'xi', 'age': 35.0, 'sex': 'male', 'grade': 75.0}
{'_id': ObjectId('5cd6e84cd422c50666051771'), 'name': '_新的标题_', 'age': 22, 'sex': 'female', 'created_at': datetime.datetime(2019, 5, 11, 23, 20, 43, 987000)}
{'_id': ObjectId('5cdc3034d422c513f136977f'), 'name': '_新的标题_', 'age': 22, 'sex': 'female', 'created

## pandas 

In [76]:
import pandas as pd

In [77]:
pd.DataFrame(list(posts.find()))

,_id,author,date,tags,text,title
0,5cdc32cad422c513f1369781,Mike,2019-05-15 15:39:05.162,"[mongodb, python, pymongo]",My first blog post!,NaN
1,5cdc332dd422c513f1369782,Mike,2019-05-15 15:41:32.257,"[mongodb, python, pymongo]",My first blog post!,NaN
2,5cdc3445d422c513f1369787,Mike,2019-05-15 15:45:51.478,"[mongodb, python, pymongo]",My first blog post!,NaN
3,5cf2339dd422c5028e526b1e,Mike,2019-06-01 08:13:17.042,"[mongodb, python, pymongo]",My first blog post!,NaN
4,5cf23447d422c5028e526b1f,Mike,2009-11-12 11:14:00.000,"[bulk, insert]",Another post!,NaN
5,5cf23447d422c5028e526b20,Eliot,2009-11-10 10:45:00.000,NaN,and pretty easy too!,MongoDB is fun
